In [1]:
import os
import torch
from DataReader import load_data, train_vaild_split
from Model import Cifar

def run_experiment(batch_size, save_interval, weight_decay, learning_rate, modeldir, use_residual, use_bn, device, save_checkpoint_model =False):
    # Set up config
    class Config:
        def __init__(self):
            self.batch_size = batch_size
            self.save_interval = save_interval
            self.weight_decay = weight_decay
            self.lr = learning_rate
            self.modeldir = modeldir
            self.use_residual = use_residual
            self.use_bn = use_bn
            self.device = device
    
    config = Config()
    
    # Load and split data
    data_dir = "dataset/cifar-10-batches-py"
    x_train, y_train, x_test, y_test = load_data(data_dir)
    x_train_new, y_train_new, x_valid, y_valid = train_vaild_split(x_train, y_train)

    # Initialize and train the model
    model = Cifar(config).to(device)
    print(f"\n--- Training with learning rate {learning_rate}, Residual: {use_residual}, BatchNorm: {use_bn} ---")
    #model.train(x_train_new, y_train_new, 100)
    model.train_test_accuracy(x_train_new, y_train_new, x_test, y_test, 100, save_checkpoint_model)

    if save_checkpoint_model:
        # Validate and test
        print("\n--- Validation ---")
        model.test_or_validate(x_valid, y_valid, [80, 90, 100])
        print("\n--- Test ---")
        model.test_or_validate(x_test, y_test, [100])

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Directory for saving models
MODEL_DIR = "models"
os.makedirs(MODEL_DIR, exist_ok=True)

# Define configurations for ResNet variations
configs = [
    {"use_residual": True, "use_bn": False, "modeldir": os.path.join(MODEL_DIR, "resnet18_no_bn")},
    {"use_residual": False, "use_bn": True, "modeldir": os.path.join(MODEL_DIR, "resnet18_no_residual")},
    {"use_residual": True, "use_bn": True, "modeldir": os.path.join(MODEL_DIR, "resnet18")},
]

# Define learning rates to test
learning_rates = [0.003, 0.01, 0.03]

# Run experiments for each configuration and learning rate
for lr in learning_rates:
    for config in configs:
        lr_modeldir = os.path.join(config["modeldir"], str(lr))
        os.makedirs(lr_modeldir, exist_ok=True)
        run_experiment(
            batch_size=128,
            save_interval=10,
            weight_decay=2e-4,
            learning_rate=lr,
            modeldir=lr_modeldir,
            use_residual=config["use_residual"],
            use_bn=config["use_bn"],
            device=device,
            save_checkpoint_model = True
        )

In [ ]:
# Run experiments for each configuration and with best learning rate
for config in configs:
    run_experiment(
        batch_size=128,
        save_interval=10,
        weight_decay=2e-4,
        learning_rate=0.003,
        modeldir=config["modeldir"],
        use_residual=config["use_residual"],
        use_bn=config["use_bn"],
        device=device,
        save_checkpoint_model = False
    )